<a href="https://colab.research.google.com/github/takatakamanbou/ML/blob/2022/ex11noteC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML ex11noteC

<img width=72 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/ML-logo.png"> [この授業のウェブページ](https://www-tlab.math.ryukoku.ac.jp/wiki/?ML/2022)


----
## $K$-平均法の実験
----

2種類のデータで K-平均法によるクラスタリングの実験をやってみましょう．



まずはいつものように準備から．



In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

K-meansクラスタリングのための関数を定義しておきます．

In [ ]:
## セントロイドの初期化
#
def initCentroid(X, centroid, seed=None):
    assert X.shape[1] == centroid.shape[1]
    K = centroid.shape[0]
    # 学習データからランダムに K 個を選択して初期セントロイドとする
    N = X.shape[0]
    idx = np.arange(N, dtype=int)
    if seed is not None:
        np.random.seed(seed)
    np.random.shuffle(idx)
    centroid[:] = X[idx[:K], :]

## データをクラスタに割り振る
#
def assignCluster(X, centroid, label):
    assert X.shape[1] == centroid.shape[1] and X.shape[0] == label.shape[0]
    K = centroid.shape[0]
    N = X.shape[0]
    sqe = 0.0
    for n in range(N):
        # 各セントロイドとの距離の二乗を計算
        d = np.sum((X[n, :] - centroid)**2, axis=1)
        # 距離最小のクラスタへ割り振る
        i = np.argmin(d)
        label[n] = i
        sqe += d[i]

    return sqe/N  # 割り振られたセントロイドとの距離の二乗の平均

## セントロイドを計算し直す
#
def updateCentroid(X, centroid, label):
    assert X.shape[1] == centroid.shape[1] and X.shape[0] == label.shape[0]
    K = centroid.shape[0]
    for ik in range(K):
        # ik 番目のクラスタに割り当てられたデータの平均をそのクラスタの新しいセントロイドとする
        centroid[ik, :] = np.mean(X[label==ik, :], axis=0)

画像のクラスタリングの実験で使う，可視化用の関数を定義しておきます．

In [ ]:
#####  データの最初の nx x ny 枚を可視化
#
def mosaicImage(dat, nx, ny, nrow=64, ncol=64, gap=4):

    # 並べた画像の幅と高さ
    width  = nx * (ncol + gap) + gap
    height = ny * (nrow + gap) + gap

    # 画像の作成
    img = np.zeros((height, width), dtype = int) + 128
    for iy in range(ny):
        lty = iy*(nrow + gap) + gap
        for ix in range(nx):
            if iy*nx+ix >= dat.shape[0]:
                break
            ltx = ix*(ncol + gap) + gap
            img[lty:lty+nrow, ltx:ltx+ncol] = dat[iy*nx+ix, :].reshape((nrow, ncol))
            
    return img


#####  セントロイドとともにデータの最初の n 枚を可視化
#
def mosaicImage2(X, centroid, index, n, nrow=64, ncol=64, gap=4):

    nx = n + 2
    ny = centroid.shape[0]
    
    # 並べた画像の幅と高さ
    width  = nx * (ncol + gap) + gap
    height = ny * (nrow + gap) + gap

    # 画像の作成
    img = np.zeros((height, width), dtype = int) + 128
    for iy in range(ny):
        lty = iy*(nrow + gap) + gap
        ix = 0
        ltx = ix*(ncol + gap) + gap
        img[lty:lty+nrow, ltx:ltx+ncol] = centroid[iy, :].reshape((nrow, ncol))
        dat = X[index == iy, :]
        for ix in range(2, nx):
            jx = ix - 2
            if jx >= dat.shape[0]:
                break
            ltx = ix*(ncol + gap) + gap
            img[lty:lty+nrow, ltx:ltx+ncol] = dat[jx, :].reshape((nrow, ncol))
            
    return img


---
### 3科目の試験の得点

「数学」「物理」「情報」の3科目の試験の点数を100人分集めたデータをクラスタリングしてみましょう．

In [ ]:
# 数物情データを入手
! wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/PIP/mpi100-mac.csv
dfMPI = pd.read_csv('mpi100-mac.csv', index_col=0)
datMPI = dfMPI.to_numpy().astype(float)
dfMPI

In [ ]:
## 数物情データの K-means クラスタリング

K = 3  # クラスタ数
nitr = 10  # K-means 法の繰り返し回数

X = datMPI
N, D = X.shape
centroid = np.empty((K, D)) # セントロイド
label = np.empty(N, dtype=int) # 各学習データの所属するセントロイドの番号

# セントロイドを初期化
initCentroid(X, centroid)  

print('#itr  msqe    Nk')
for i in range(nitr):
    # データを各クラスタに割り振る
    msqe = assignCluster(X, centroid, label)
    # 各クラスタに割り振られたデータの数と，誤差の値を出力
    Nk = np.empty(K, dtype=int)
    for ik in range(K):
        Nk[ik] = np.sum(label == ik)
    print(f'{i}    {msqe:.2f}   {Nk}')    
    # セントロイドを更新
    updateCentroid(X, centroid, label)

# セントロイドの値を出力
for ik in range(K):
    print(f'クラスタ {ik} のセントロイド:  ({centroid[ik, 0]:.1f}, {centroid[ik, 1]:.1f}, {centroid[ik, 2]:.1f})')

#### ★★★ やってみよう ★★★

- クラスタリングを実行して得られる結果を観察しよう．実行のたびに初期値が変わり，学習の結果も変わるので，何度か実行し直してみよう．
- 3つのクラスタに分けられたそれぞれの集団の得点の傾向を，セントロイドの値から考察してみよう．「他の集団に比べて○○の点が高い／低い」，etc.

---
### 猫顔画像のクラスタリング

猫の顔画像をクラスタリングしてみましょう．
この実験で使っている猫画像は，他の目的に使ってはいけません．

In [ ]:
# 猫画像データを入手
! wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/PIP/cat131.npz
cat = np.load('cat131.npz')['cat131']
print(cat.shape) # 131枚，ひとつの画像は 64 x 64 = 4096 画素のグレイスケール画像

In [ ]:
print('### 猫画像 ###')
print(f'データ数 x 次元数 = {cat.shape[0]} x {cat.shape[1]}')

# 最初の 16 枚を表示 
img = mosaicImage(cat, 4, 4)
plt.axis('off')
plt.imshow(img, cmap = 'gray')
plt.show()   

学習を実行し，結果を可視化させてみましょう．

In [ ]:
## 猫画像データの K-means クラスタリング

K = 3  # クラスタ数
nitr = 10  # K-means 法の繰り返し回数

X = cat
N, D = X.shape
centroid = np.empty((K, D)) # セントロイド
label = np.empty(N, dtype=int) # 各学習データの所属するセントロイドの番号

# セントロイドを初期化
initCentroid(X, centroid)  

print('#itr  msqe    Nk')
for i in range(nitr):
    # データを各クラスタに割り振る
    msqe = assignCluster(X, centroid, label)
    # 各クラスタに割り振られたデータの数と，誤差の値を出力
    Nk = np.empty(K, dtype=int)
    for ik in range(K):
        Nk[ik] = np.sum(label == ik)
    print(f'{i}    {msqe:e}   {Nk}')    
    # セントロイドを更新
    updateCentroid(X, centroid, label)

# クラスタ毎のセントロイドと所属データを可視化
#     各行が一つのクラスタに対応，左端がセントロイド，右の画像はそのクラスタに所属する画像の一部
img = mosaicImage2(cat, centroid, label, 8)
plt.figure(figsize=(10,10))
plt.axis('off')
plt.imshow(img, cmap = 'gray')
plt.show()
s = '''
クラスタ毎のセントロイドと所属データを可視化
　各行が一つのクラスタに対応，左端がセントロイド，右の画像はそのクラスタに所属する画像の一部
'''
print(s)

#### ★★★ やってみよう★★★

上記では，猫画像を K = 3 の K-means法でクラスタリングする実験を行っています．K をいろいろ変えて実行し直して，結果を観察しよう．K-meansクラスタリングの結果は初期値に依存するので，実行するたびに結果が変わります．